# Parallel beam

-----------------------------------------------------------------------------------------------------------------------

<u>*The material provided in this notebook can be freely used and modified for educational purposes only. Please cite any content of the notebook as follows:*</u>

- *Panetta D, Camarlinghi N. 3D Image Reconstruction for CT and PET : A Practical Guide with Python. CRC Press; 2020. Available from: https://www.taylorfrancis.com/books/9780429270239*

*For questions, notifications of bugs, or even just for feedback, please contact the authors directly (daniele.panetta@ifc.cnr.it; niccolo.camarlinghi@gmail.com)*

-----------------------------------------------------------------------------------------------------------------------

In [1]:
import sys
import numpy as np
sys.path.append("../") # this to be able to include all the object contained in the modules
from scipy.ndimage import gaussian_filter
from Misc.Utils import Unpickle,ReadImage
from Misc.Preview import Visualize3dImage
import matplotlib.pyplot as plt
# set the default size of all the plots
# if images are too big or too small try to play whit these numbers
plt.rcParams['figure.figsize'] = [10, 10] 
from Algorithms.MLEM import MLEM
from Algorithms.SinogramGenerator_3D import SinogramGenerator_3D
from Geometry.ExperimentalSetupCT_3D import ExperimentalSetupCT_3D, Mode, DetectorShape
from Misc.DataTypes import voxel_dtype


ModuleNotFoundError: No module named 'imageio'

### Create a CT experimental setup

Now, we are going to create an experimental setup along with its geometry, by creating an instance of the ```ExperimentalSetupCT``` class. In this example, we are about to reconstruct 10 slices of the phantom, each with 100x100 pixels in the $x,y$ plane. The number of voxels in each direction is a derived parameter, calculated on top of ```image_matrix_size_mm``` (i.e., the numpy vector containing the lengths in mm of each side of the reconstruction volume) and ```voxel_size_mm``` (i.e, the numpy vector containing the lengths in mm of each side of the voxel).

In [ ]:
# create CT experimental setup
my_experimental_setup = ExperimentalSetupCT_3D()
my_experimental_setup.mode = Mode.PARALLELBEAM
my_experimental_setup.detector_shape=DetectorShape.PLANAR
my_experimental_setup._detector_number = 3
# number of sensitive elements 
my_experimental_setup.pixels_per_slice_nb=100
my_experimental_setup.detector_slice_nb=100
my_experimental_setup.slice_pitch_mm=1
# number of rotation of the gantry
my_experimental_setup.gantry_angles_nb = 1
# range of the rotation
my_experimental_setup.angular_range_deg = 360
# fov size in mm 
my_experimental_setup.image_matrix_size_mm = np.array([100,100,100])
# voxel size in mm
my_experimental_setup.voxel_size_mm = np.array([1,1,1])

# compute the geometry
my_experimental_setup.Update()
print(my_experimental_setup.GetInfo())

### Display the experimental setup

The ```Draw()``` method of the ```ExperimentalSetupCT``` class will allow us to display the geometry (source and detector position(s), as well as the reconstruction matrix). When using ```use_jupyter=1```, only a static figure will be displayed in the notebook itself. Otherwise, the user can obtain an interactive display of the same geometry by putting ```use_jupyter=0``` as the first argument. The second argument is the position of the camera in the static (```use_jupyter=1```) mode.

In [ ]:
my_experimental_setup.Draw(use_jupyter=0,camera_pos_mm=(0,-400,100))

### Load the image used to generate the sinogram

We are now ready to lead the example dataset provided, a voxelised 3D version of the Shepp-Logan head phantom (defined in the great PhD thesis of Enrik Turbell at Linkopings University - https://people.csail.mit.edu/bkph/courses/papers/Exact_Conebeam/Turbell_Thesis_FBP_2001.pdf). Upon loading the dataset, made up by a (rather coarse) grid of 100$^3$ voxels, we will select just 10 consecutive slices around the plane containing the well known triplet of small low-contrast details.

In [ ]:
input_img = np.load('../Data/cilinder.npz')['matrix'].astype(np.float64)
print('Image shape: ', input_img.shape)

n = (3,3,3)

#input_img = gaussian_filter(input_img, sigma = n, order = 0)

fig, ax = plt.subplots(2, 3, figsize=(8,8))
ax[0,0].imshow(input_img[50,:,:], vmin = 0, vmax=1)
ax[0,1].imshow(input_img[:,50,:], vmin = 0, vmax=1)
im = ax[0,2].imshow(input_img[:,:,50], vmin = 0, vmax=1)
fig.colorbar(im, ax=[ax[0,0], ax[0,1], ax[0,2]], orientation = 'horizontal')
ax[1,0].plot(input_img[50,50,:])
ax[1,1].plot(input_img[:,50,50])
im = ax[1,2].plot(input_img[50,:,50])


In [ ]:
#Carica le proiezioni

### Generate and display the sinogram

The phantom data stored in the numpy 3D array ```img``` is now ready to be forward projected using the Siddon method (see Chapter 5 of the book). Fist of all, we must create an instance of the ```SinogramGenerator``` class, which takes the experimental setup as the only argument. Then, a sinogram object ```sino``` is created with the method ```GenerateObjectSinogram()``` of the ```SinogramGenerator``` class, taking the voxelised array as input. The ```transpose_image=1``` argument is required internally, in order to keep coherence between the array axes in the implementation of the Siddon algorithm. 

In most PC, the forward projection step in this example shuold last no longer than 1 minute or so. It will increase if the projection grid is made finer, or more slices are put in the computation.

In [ ]:
s=SinogramGenerator_3D(my_experimental_setup)
sino_list=s.GenerateObjectSinogram(input_img,transponse_image=1)

for s in sino_list: 
    print('sino_list[s].shape: ' , s._data.shape)

Let's now visualize the resulting sinogram, slice by slice, again by calling the ```Visualize3dImage``` method. As it can be seen, the angular coordinate is displayed in the horizonal axis and the radial coordinate in the vertical axis. The number of bins in both directions is defined in the experimental setup.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 6))  
plt.subplots_adjust(wspace=0.4)  

im = ax[0].imshow(np.transpose(sino_list[0]._data[:,0,:]), extent=[-50, 50, -50, 50])
ax[0].set_xticks(np.linspace(-50, 50, 5))  
ax[0].set_yticks(np.linspace(-50, 50, 5))  

im = ax[1].imshow(np.transpose(sino_list[2]._data[:,0,:]), extent=[-50, 50, -50, 50])
ax[1].set_xticks(np.linspace(-50, 50, 5))  
ax[1].set_yticks(np.linspace(-50, 50, 5)) 

im = ax[2].imshow(sino_list[1]._data[:,0,:], extent=[-50, 50, -50, 50])
ax[2].set_xticks(np.linspace(-50, 50, 5))  
ax[2].set_yticks(np.linspace(-50, 50, 5)) 

cbar = fig.colorbar(im, ax=ax, orientation='horizontal', fraction=0.3, pad=0.15)
cbar.set_label('Intensity')  
plt.show()

In [ ]:
projections1 = np.concatenate(np.concatenate(np.transpose(sino_list[0]._data, axes=(2,1,0)), axis=1))
projections2 = np.concatenate(np.concatenate(np.transpose(sino_list[1]._data, axes=(2,1,0)), axis=1))
projections3 = np.concatenate(np.concatenate(np.transpose(sino_list[2]._data, axes=(2,1,0)), axis=1))

projections = np.concatenate((projections1, projections2))

print('projections1.shape, projections2.shape, projections3.shape, projections.shape:, ', projections1.shape, projections2.shape, projections3.shape, projections.shape)
projections = np.concatenate((projections, projections3))

In [ ]:
algorithm="MLEM"
# number of iterations 
niter=10
# when use using MLEM or OSEM remember to set this value to !=0 
initial_value=1

In [ ]:
it = eval( algorithm+ "()")
it.SetExperimentalSetup(my_experimental_setup)
it.SetNumberOfIterations(niter)
#it.SetNumberOfSubsets(nsubsets)
it.SetProjectionData(projections)
# start with a initial_guess filled image
initial_guess=np.full(it.GetNumberOfVoxels(),initial_value, dtype=voxel_dtype) 
it.SetImageGuess(initial_guess)
# uncomment this line to save images to disk
#it.SetOutputBaseName(basename)
output_img = it.Reconstruct()

In [ ]:
np.savez('../Data/cilinder_reconstruction_parallelbeam.npz', matrix=output_img)

In [ ]:
print(output_img.shape)
index_x = 50
index_y = 50
index_z = 50

vmin, vmax = 0.,1.
fig, ax = plt.subplots(1, 3, figsize=(12,8))
im=ax[0].imshow(output_img[index_x,:,:], vmin=vmin, vmax=vmax)
im=ax[1].imshow(output_img[:,index_y,:], vmin=vmin, vmax=vmax)
im=ax[2].imshow(output_img[:,:,index_z], vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=[ax[0], ax[1], ax[2]], orientation = 'horizontal')

vmin, vmax= -0.01, 0.01
fig, ax = plt.subplots(1, 3, figsize=(12,12))
ax[0].imshow(output_img[index_x,:,:]-input_img[index_x, :, : ], vmin=vmin, vmax=vmax, cmap='RdBu')
ax[1].imshow(output_img[:,index_y,:] - input_img[:, index_y,:], vmin=vmin, vmax=vmax, cmap='RdBu')
im = ax[2].imshow(output_img[:,:,index_z] - input_img[:,:,index_z], vmin=vmin, vmax=vmax, cmap='RdBu')
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, ax=[ax[0], ax[1], ax[2]], orientation = 'horizontal')

In [ ]:
output_img

### Run the FBP algorithm and display the reconstructed image

Let us now reconstruct the sinograms. The theory behind FBP reconstruction is described in Chapter 3 of the book. First, we create a ```FBP``` object called ```f```, and we assign the previously generated sinogram object to its member ```f.sinogram```. The type of interpolation to be used in the backprojection step is stored in ```f.interpolator``` (see comments in the code).

In [ ]:
f=FBP()
f.sinogram=sino
# this is the interpolation for the backprojection 
# available options are : "linear","nearest","zero","slinear","quadratic","cubic"
# see for https://docs.scipy.org/doc/scipy/reference/generated/szerocipy.interpolate.interp1d.html parameter: kind
# for an explanation of the interpolation parameters  
f.interpolator='cubic'


Ready? The next line will make the FBP reconstruction to start, finally! Some text message will appear to let the user know the progress status of the reconstruction during the computation, even though this example shuold run quite fast in most PC's.

In [ ]:
f.Reconstruct()


If you can read ```Reconstruction done``` in the log text above, then you're done. All the slices have been reconstruction, so let's display them now. The result is stored in ```f._image```, so let's put this object as argument in the ```Visualize3dImage()``` method.

In [ ]:
# We are rotating 180° the image before displaying it, this is required due to 
# the internal implementation of the backprojection function in order to make it
# perfectly comparable with the original one.
f._image = np.rot90(f._image,2)
Visualize3dImage(f._image)

In this FBP demo in parallel beam geometry, we used the DAPHNE framework as a high-level, easy to use API to run all the tasks relevant for the reconstruction. Some other notebook in this folder wil avoid DAPHNE, so the user will see all the hard computation steps at "low"-level. 
We can now explore a bit more some relevant building block. For instance, let's see the shape of the ramp filter in the spatial and frequency domain.
As explained in Chapter 3 of the book, the frequency response of the ramp filter, $H(\nu)$ is intrinsically a 1D function of the spatial frequency. Indeed, to make things more "pythonic", in our code within DAPHNE we are storing repeated copies of the 1D filter function in a multidimensional array called ```_Hm``` (where 'm' stands for 'matrix'). With this trick, the filtration in the frequency domain of the sinogram is just performed by multiplying (internally, in the ```f.Reconstruct()``` method) the FFT of the sinogram of each slice with the ```_Hm``` function.

In [ ]:
plt.figure()
plt.plot(f._Hm[:,0])
plt.title("Ramp filter freq. response")


In the Figure above, the negative frequencies appear in the second half of the plot (i.e., for $N_{bin}/2 \leq k < N_{bin}/2 - 1$). Let's now display also the impulse response of the filter, $h(x')$.

In [ ]:
plt.figure()
plt.title("Ramp filter impulse response")
plt.plot(f._h)

As a last step in this exercise, let's visualize the filtered sinogram stored in ```f._filtered_sinogram```:

In [ ]:
Visualize3dImage(f._filterderd_sinogram,2)